# Setup Test Data for Curation Framework MVP

This notebook initializes the `ltc_insurance` catalog and schemas, and populates the `raw_data_layer` with sample data for the MVP.

## 1. Setup Catalog and Schemas

In [ ]:
spark.sql("CREATE CATALOG IF NOT EXISTS ltc_insurance")
spark.sql("CREATE SCHEMA IF NOT EXISTS ltc_insurance.raw_data_layer")
spark.sql("CREATE SCHEMA IF NOT EXISTS ltc_insurance.curated_data_layer")

print("Catalog and schemas created.")

## 2. Define Sample Data

We define the sample data directly here to avoid dependency on external file uploads.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, DateType, TimestampType

# Claimants Data
claimants_data = [
    {"claimant_id":1,"source_system":"crm","first_name":"John","last_name":"Doe","date_of_birth":"1975-02-10","address":"123 Main St","city":"Boston","state":"MA","zip_code":"02110","phone":"6175551000","email":"john.doe@example.com","care_level":"home_health","ingestion_ts":"2024-01-15T10:00:00Z","_kafka_offset":100},
    {"claimant_id":1,"source_system":"crm","first_name":"John","last_name":"Doe","date_of_birth":"1975-02-10","address":"123 Main St","city":"Boston","state":"MA","zip_code":"02110","phone":"6175551000","email":"john.d@example.com","care_level":"home_health","ingestion_ts":"2024-01-15T10:05:00Z","_kafka_offset":101},
    {"claimant_id":2,"source_system":"admin","first_name":"Alice","last_name":"Smith","date_of_birth":"1980-08-20","address":"45 River Rd","city":"Hartford","state":"CT","zip_code":"06103","phone":"8605552000","email":"alice.smith@example.com","care_level":"assisted_living","ingestion_ts":"2024-01-15T10:07:00Z","_kafka_offset":50}
]

# Policies Data
policies_data = [
    {"policy_id":100,"source_system":"crm","policy_status":"active","coverage_type":"ltc","daily_benefit_amount":200,"benefit_period":"3y","elimination_period":"90d","premium_amount":1200,"ingestion_ts":"2024-01-15T09:55:00Z","_kafka_offset":10},
    {"policy_id":100,"source_system":"crm","policy_status":"lapsed","coverage_type":"ltc","daily_benefit_amount":200,"benefit_period":"3y","elimination_period":"90d","premium_amount":1200,"ingestion_ts":"2024-01-15T10:20:00Z","_kafka_offset":11},
    {"policy_id":101,"source_system":"admin","policy_status":"active","coverage_type":"ltc","daily_benefit_amount":180,"benefit_period":"2y","elimination_period":"60d","premium_amount":900,"ingestion_ts":"2024-01-15T10:10:00Z","_kafka_offset":5}
]

# Claims Data
claims_data = [
    {"claim_id":5000,"source_system":"crm","claimant_id":1,"policy_id":100,"claim_status":"OPEN","claim_amount":1500.75,"service_date":"2024-01-14","submitted_at":"2024-01-15T09:00:00Z","ingestion_ts":"2024-01-15T10:12:00Z","_kafka_offset":200},
    {"claim_id":5001,"source_system":"admin","claimant_id":2,"policy_id":101,"claim_status":"PENDING","claim_amount":800.00,"service_date":"2024-01-13","submitted_at":"2024-01-15T09:05:00Z","ingestion_ts":"2024-01-15T10:15:00Z","_kafka_offset":210}
]


## 3. Create Tables and Insert Data

We create the tables and insert the data. We use `overwrite` mode to ensure a clean state for testing.

In [ ]:
# Helper function to create and load table
def create_and_load_table(table_name, data):
    df = spark.createDataFrame(data)
    
    # Ensure timestamp columns are cast correctly if they are strings
    if "ingestion_ts" in df.columns:
        df = df.withColumn("ingestion_ts", df["ingestion_ts"].cast("timestamp"))
    if "submitted_at" in df.columns:
        df = df.withColumn("submitted_at", df["submitted_at"].cast("timestamp"))
    if "date_of_birth" in df.columns:
        df = df.withColumn("date_of_birth", df["date_of_birth"].cast("date"))
    if "service_date" in df.columns:
        df = df.withColumn("service_date", df["service_date"].cast("date"))

    full_table_name = f"ltc_insurance.raw_data_layer.{table_name}"
    print(f"Creating and loading {full_table_name}...")
    
    # Write to table
    df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
    print(f"Loaded {df.count()} records into {full_table_name}")

# Create and load tables
create_and_load_table("claimants_streaming", claimants_data)
create_and_load_table("policies_streaming", policies_data)
create_and_load_table("claims_streaming", claims_data)

print("All test data loaded successfully.")